In [1]:
import pandas as pd
import helpers
from importlib import reload
import random
import models
import rocket
import numpy as np
import os
from datetime import datetime
from datetime import datetime, timezone

2024-06-14 09:28:57.625354: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 09:29:01.959465: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Load pressure drop list

In [2]:
reload(helpers)

pressuredrops = pd.read_csv('allPdrops_ordered.csv').sort_values(by=' SOL ')

unique_sols = pressuredrops[' SOL '].unique()
missing_sols = []
for sol in unique_sols:
    file_path = f'seisdata_pq/sol_{sol}_seisdata.parquet'
    if not os.path.isfile(file_path):
        pressuredrops = pressuredrops[pressuredrops[' SOL '] != sol]
        missing_sols.append(sol)

pressuredrops[' YYYY-MM-DDTHH:MM:SS.sss '] = pressuredrops[' YYYY-MM-DDTHH:MM:SS.sss '].str.strip()
pressuredrops[' YYYY-MM-DDTHH:MM:SS.sss '] = pd.to_datetime(pressuredrops[' YYYY-MM-DDTHH:MM:SS.sss '], format='%Y-%jT%H:%M:%S.%fZ')

pressure_threshold = -0

filtered_drops = pressuredrops[pressuredrops['_DROP_ '] < pressure_threshold]
print(filtered_drops.describe())

            _DROP_        _LTST_           SOL   \
count  11560.000000  11560.000000  11560.000000   
mean      -0.611105     12.289125    460.845329   
min       -9.183000      8.001000     65.000000   
25%       -0.603000     10.890500    309.000000   
50%       -0.450000     12.193500    434.500000   
75%       -0.389000     13.601250    639.000000   
max       -0.345000     21.058000    861.000000   
std        0.511879      1.839304    191.826052   

            YYYY-MM-DDTHH:MM:SS.sss          RATIO  
count                          11560  11560.000000  
mean   2020-03-14 06:10:28.823963136      0.989166  
min       2019-02-01 11:13:59.915000      0.502000  
25%    2019-10-10 04:44:05.309999872      1.000000  
50%    2020-02-16 02:58:21.841500160      1.000000  
75%    2020-09-13 08:57:15.142499840      1.000000  
max       2021-04-29 12:06:01.663000      1.000000  
std                              NaN      0.044255  


### Feature List

In [3]:
single_features = [
    #'Time of day',
    #'Time of year',
    #'Time since last pDrop',
    #'Drop width',
    #'Max power',
    #'Max vs average'
]

ts_features = [
    #'Total',
    #'Gradient'
]

feature_list = single_features + ts_features

### Extract dataset

In [4]:
reload(helpers)
window_size = 35
features = 3 + len(ts_features)
power_threshold = 1.148e-8
samples_per_drop = 5
pos_sample_spacing = 5
sol_range = [14, 862]
test_sols = helpers.generate_test_sols(sol_range=sol_range, percentage=0.1)
pos_df, pDrops_below_thresh, pressuredrops, pDrops_out_of_range, pos_thresh_dict = helpers.construct_pos_dataset(feature_list=feature_list, sol_range=sol_range, test_sols=test_sols, drop_list=filtered_drops, power_threshold=power_threshold, window_size = window_size, samples_per_drop = samples_per_drop, pos_sample_spacing= pos_sample_spacing, verbose=1, drop_magnitude=False)
pos_length = len(pos_df)

Processing SOL 14
Processing SOL 15
Processing SOL 16
Processing SOL 17
Processing SOL 18
Processing SOL 19
Processing SOL 20
Processing SOL 21
Processing SOL 22
Processing SOL 23
Processing SOL 24
Processing SOL 25
Processing SOL 26
Processing SOL 27
Processing SOL 28
Processing SOL 29
Processing SOL 30
Processing SOL 31
Processing SOL 32
Processing SOL 33
Processing SOL 34
Processing SOL 35
Processing SOL 36
Processing SOL 37
Processing SOL 38
Processing SOL 39
Processing SOL 40
Processing SOL 41
Processing SOL 42
Processing SOL 43
Processing SOL 44
Processing SOL 45
Processing SOL 46
Processing SOL 47
Processing SOL 48
Processing SOL 49
Processing SOL 50
Processing SOL 51
Processing SOL 52
Processing SOL 53
Processing SOL 54
Processing SOL 55
Processing SOL 56
Processing SOL 57
Processing SOL 58
Processing SOL 59
Processing SOL 60
Processing SOL 61
Processing SOL 62
Processing SOL 63
Processing SOL 64
Processing SOL 65
Processing SOL 66
Processing SOL 67
Processing SOL 68
Processing

In [5]:
reload(helpers)
noise_range = [power_threshold/8, power_threshold/4]
neg_df, neg_pDrops_excluded, neg_below_thresh, noise_df = helpers.construct_neg_samples(feature_list, sol_range, test_sols, pressuredrops, power_threshold, sample_size=window_size, interval=35, verbose=1, noise_range=noise_range)
neg_length = len(neg_df)

Processing SOL 14
Processing SOL 15
Processing SOL 16
Processing SOL 17
Processing SOL 18
Processing SOL 19
Processing SOL 20
Processing SOL 21
Processing SOL 22
Processing SOL 23
Processing SOL 24
Processing SOL 25
Processing SOL 26
Processing SOL 27
Processing SOL 28
Processing SOL 29
Processing SOL 30
Processing SOL 31
Processing SOL 32
Processing SOL 33
Processing SOL 34
Processing SOL 35
Processing SOL 36
Processing SOL 37
Processing SOL 38
Processing SOL 39
Processing SOL 40
Processing SOL 41
Processing SOL 42
Processing SOL 43
Processing SOL 44
Processing SOL 45
Processing SOL 46
Processing SOL 47
Processing SOL 48
Processing SOL 49
Processing SOL 50
Processing SOL 51
Processing SOL 52
Processing SOL 53
Processing SOL 54
Processing SOL 55
Processing SOL 56
Processing SOL 57
Processing SOL 58
Processing SOL 59
Processing SOL 60
Processing SOL 61
Processing SOL 62
Processing SOL 63
Processing SOL 64
Processing SOL 65
Processing SOL 66
Processing SOL 67
Processing SOL 68
Processing

In [ ]:
reload(helpers)

pos_centered = pos_df.iloc[::5, :].copy()
filtered_pos_df = pos_df.drop(pos_df.index[::5])

noise_df['LTST'] = noise_df.index.map(helpers.datetime_to_mlst)
noise_df = noise_df[(noise_df['LTST'] > 6) & (noise_df['LTST'] < 18)]
filtered_noise_df = noise_df.drop(columns='LTST')
multiplier = (len(neg_df)-len(pos_centered))/len(filtered_pos_df)
synthetic_samples = helpers.create_synthetic_samples(filtered_pos_df, filtered_noise_df, multiplier=multiplier)
pos_df = pd.concat([pos_centered, synthetic_samples], ignore_index=False)

### Concat features, create labels

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

features_df = pd.concat([pos_df, neg_df], ignore_index=False)

print(len(features_df))

ratio = len(neg_df) / len(pos_df)

positive_labels = pd.Series([1] * len(pos_df))
negative_labels = pd.Series([0] * len(neg_df))

labels_df = pd.concat([positive_labels, negative_labels], ignore_index=True)
labels_df.index = features_df.index


rows_with_nan = features_df.isnull().any(axis=1)
indices_with_nan = features_df.index[rows_with_nan]
labels_df.drop(indices_with_nan, inplace=True)
features_df.drop(indices_with_nan, inplace=True)
features_df.columns = features_df.columns.astype(str)

assert len(features_df) == len(labels_df), f"Mismatch in length of features and labels dataframes, features length {len(features_df)}, labels length {len(labels_df)}"
print(ratio)

feature_scaler = StandardScaler()
features_scaled = pd.DataFrame(feature_scaler.fit_transform(features_df), columns=features_df.columns, index=features_df.index)
if 'Time of day' in feature_list:
    mlst_series = features_scaled.index.to_series().apply(lambda x: helpers.datetime_to_mlst(x))
    features_scaled[['sin_time', 'cos_time']] = mlst_series.apply(helpers.cyclical_encode_time).apply(pd.Series)
if 'Time of year' in feature_list:
    features_scaled[['sin_martian_time_of_year', 'cos_martian_time_of_year']] = features_scaled.index.to_series().apply(helpers.cyclical_encode_martian_time_of_year).apply(pd.Series)

if 'Time since last pDrop' in feature_list:
    pressuredrops_time = pressuredrops[[' YYYY-MM-DDTHH:MM:SS.sss ']].sort_values(' YYYY-MM-DDTHH:MM:SS.sss ')
    features_scaled = features_scaled.sort_index()
    features_scaled = pd.merge_asof(features_scaled, pressuredrops_time, left_index=True, right_on=' YYYY-MM-DDTHH:MM:SS.sss ', direction='backward')
    features_scaled['time since last pDrop'] = features_scaled.index - features_scaled[' YYYY-MM-DDTHH:MM:SS.sss ']
    features_scaled['time since last pDrop'] = features_scaled['time since last pDrop'].dt.total_seconds().astype(int)
    features_scaled['time since last pDrop'] = features_scaled['time since last pDrop'].clip(upper=86400)
    features_scaled.drop(' YYYY-MM-DDTHH:MM:SS.sss ', axis=1, inplace=True)

X_train, X_eval, y_train, y_eval = train_test_split(features_scaled, labels_df, test_size=0.2, random_state=123)
X_val, X_test, y_val, y_test = train_test_split(X_eval, y_eval, test_size=0.5, random_state=123)
X_train_final = pd.concat([X_train, X_val])
y_train_final = pd.concat([y_train, y_val])

train_data = [X_train, y_train, X_val, y_val]

### XGBoost

In [ ]:

from sklearn.metrics import classification_report
reload(models)

adjust_ratio = True

best_xgb_clf = models.train_xgboost(data=[df.copy() for df in train_data], bayes_search=False, verbose=1, ratio=ratio, adjust_ratio=adjust_ratio)
y_pred = best_xgb_clf.predict(X_test)
class_report_xgb = classification_report(y_test, y_pred)
print("\nClassification Report:\n", class_report_xgb)

best_xgb_clf.fit(X_train_final, y_train_final, eval_set=[(X_train_final, y_train_final),(X_test, y_test)], eval_metric=['error', 'logloss'])


### LSTM

In [ ]:
from sklearn.metrics import classification_report
reload(models)
adjust_ratio = False

lstm_eval_model, lstm_final_model = models.train_lstm_test(data=[df.copy() for df in train_data], test_data = [X_test.copy(), y_test.copy()], window_size=window_size, num_features=features, gridsearch=False, verbose=1, ratio=ratio, adjust_ratio=adjust_ratio)

y_pred = (models.predict_lstm(lstm_eval_model, X_test.copy(), window_size, features) > 0.5).astype("int32")

class_report_lstm = classification_report(y_test, y_pred)
print("\nClassification Report:\n", class_report_lstm)

## Generate Rocket features

In [ ]:
reload(rocket)
data, test_data, extra_features = rocket.prepare_rocket_data(data=[df.copy() for df in train_data], test_data=[X_test.copy(), y_test.copy()])
X_train_rocket, X_val_rocket, X_test_rocket, rocket_model = rocket.get_rocket_features(data=data, test_data=test_data, window_size=window_size, num_features=features)

if len(extra_features[0] > 0):
    X_train_rocket = np.hstack((X_train_rocket, extra_features[0].values))
    X_val_rocket = np.hstack((X_val_rocket, extra_features[1].values))
    X_test_rocket = np.hstack((X_test_rocket, extra_features[2].values))

### XGBoost head

In [ ]:
'''
reload(models)

best_xgb_rocket_clf = models.train_xgboost(bayes_search=False, verbose=1, ratio=ratio, adjust_ratio=True)
y_pred = best_xgb_rocket_clf.predict(X_test_rocket)

class_report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", class_report)

best_xgb_rocket_clf.fit(np.concatenate((X_train_rocket, X_val_rocket)), y_train_final, eval_set=[(X_test_rocket, y_test)])
'''

### Ridge Classifier Head

In [ ]:
'''
from sklearn.linear_model import RidgeClassifierCV

ridge = RidgeClassifierCV(alphas=np.logspace(-8, 8, 17))
ridge.fit(np.concatenate((X_train_rocket, X_val_rocket)), y_train_final)

y_pred = ridge.predict(X_test_rocket)
print(classification_report(y_test, y_pred))

ridge.fit(np.concatenate((X_train_rocket, X_val_rocket, X_test_rocket)), np.concatenate((y_train_final, y_test)))
'''

### Logistic Regression Head

In [ ]:
from sklearn.metrics import classification_report
reload(models)
logistic_clf = models.train_logistic_regression([X_train_rocket, y_train, X_val_rocket, y_val], search_best_params=False, verbose=1)

y_pred = logistic_clf.predict(X_test_rocket)

class_report_rocket = classification_report(y_test, y_pred)
print("\nClassification Report:\n", class_report_rocket)

logistic_clf.fit(np.concatenate((X_train_rocket, X_val_rocket, X_test_rocket)), np.concatenate((y_train_final, y_test)))

## Evaluate on actual data

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

save = True
eval_mode = 'year 2 preds'
sample_interval = 1

if eval_mode == 'train_eval':
    eval_sols = [num for num in range(sol_range[0], sol_range[1]) if num not in test_sols]

elif eval_mode == 'test_eval':
    eval_sols = test_sols

elif eval_mode == 'year 2 preds':
    eval_sols = list(range(862, 1441))

rocket_model_params=None
reload(helpers)
model_type = 'xgb'
if model_type == 'xgb': 
    model = best_xgb_clf
    class_report = class_report_xgb
elif model_type == 'lstm':
    model = lstm_final_model
    class_report = class_report_lstm
elif model_type == 'xgb_rocket':
    model = best_xgb_rocket_clf
elif model_type == 'classic rocket':
    model = ridge
elif model_type == 'logistic rocket':
    model = logistic_clf
    class_report = class_report_rocket
    rocket_model_params = rocket_model

raw_predictions, total_pdrops_excluded, total_below_thresh = helpers.predict(feature_list=feature_list, test_sols=eval_sols, best_clf=model, model_type=model_type, feature_scaler=feature_scaler, pressuredrops=pressuredrops, power_threshold=power_threshold, sample_interval=sample_interval, sample_size=window_size, features=features, rocket_model=rocket_model_params)


## Ensemble Predictions

In [ ]:
import joblib
import json
reload(helpers)

sample_interval = 1
window_size = 35
features = 3
power_threshold = 1.148e-8
eval_mode = 'test_eval'

feature_lists = [['Time of day', 'Time of year'], ['Time of day', 'Max vs average'], ['Time of day']]
model_types = ['xgb', 'lstm', 'logistic rocket']
xgb = joblib.load('./xgboost_feature_selection/xgb_0_1.148e-08_Time of day_Time of year/model.joblib')
lstm = joblib.load('./lstm_feature_selection/lstm_0_1.148e-08_Time of day_Max vs average/model.joblib')
log_clf = joblib.load('./rocket_feature_selection/logistic rocket_0_1.148e-08_Time of day/model.joblib')
rocket_model = joblib.load('./rocket_feature_selection/logistic rocket_0_1.148e-08_Time of day/rocket_model.pkl')
with open('./xgboost_feature_selection/xgb_0_1.148e-08_Time of day_Time of year/positive_predictions.json', 'r') as json_file:
    positive_predictions = json.load(json_file)
eval_sols = [int(key) for key in positive_predictions.keys()]
xgb_scaler = joblib.load('xgboost_feature_selection/xgb_feature_scaler.joblib')
lstm_scaler = joblib.load('lstm_feature_selection/lstm_feature_scaler.joblib')
rocket_scaler = joblib.load('rocket_feature_selection/rocket_feature_scaler.joblib')

models = [xgb, lstm, log_clf]
feature_scalers = [xgb_scaler, lstm_scaler, rocket_scaler]

raw_predictions, total_pdrops_excluded, total_below_thresh = helpers.predict(feature_lists=feature_lists, test_sols=eval_sols, best_clfs=models, model_types=model_types, feature_scalers=feature_scalers, pressuredrops=pressuredrops, power_threshold=power_threshold, sample_interval=sample_interval, sample_size=window_size, features=features, rocket_model=rocket_model, ensemble=True)

## Refine Predictions

In [ ]:
reload(helpers)

if eval_mode != 'year 2 preds':
    best_F1 = 0
    best_match_threshold = 0
    for match_threshold in range(0,35):
        result = helpers.evaluate_predictions(raw_predictions, eval_sols, pressuredrops, match_threshold=match_threshold, sample_size=window_size)
        F1 = result['f1_score']
        print(F1, result['recall'], result['precision'])
        if F1 > best_F1:
            best_F1 = F1
            eval_result = result
            best_match_threshold = match_threshold
    positive_predictions = eval_result['predictions']

else:
    best_match_threshold = 22
    eval_result = helpers.evaluate_predictions(raw_predictions, eval_sols, pressuredrops, match_threshold=best_match_threshold, sample_size=window_size)
    positive_predictions = eval_result['predictions']

print("=" * 30)
print("Evaluation Metrics")
print("=" * 30)
print(f"True Positives (TP): {eval_result['TP']}")
print(f"False Negatives (FN): {eval_result['FN']}")
print(f"False Positives (FP): {eval_result['FP']}")
print(f"Unique Events Matched: {eval_result['unique_events_matched']}")
print(f"Precision: {eval_result['precision']:.4f}")
print(f"Recall: {eval_result['recall']:.4f}")
print(f"F1 Score: {eval_result['f1_score']:.4f}")
print("=" * 30)

## Log Results

In [11]:
import joblib
reload(helpers)
if save == True:
    feature_names = '_'.join(feature_list)
    directory_name = f"training_runs/{eval_mode}_{model_type}_{pressure_threshold}_{power_threshold}_{feature_names}"
    log_string = helpers.log_results(pressure_threshold, power_threshold, window_size, samples_per_drop, pos_sample_spacing, sol_range, pos_length, len(pDrops_below_thresh), neg_length, neg_below_thresh, adjust_ratio, model, model_type, class_report, eval_mode, sample_interval, best_match_threshold, eval_result, feature_list)
    helpers.save(raw_predictions, positive_predictions, log_string, model, directory_name)
    if model_type == 'logistic rocket':
        joblib.dump(rocket_model, f'./{directory_name}/rocket_model.pkl')